In [1]:
# ============================================================
# CELL 1: IMPORTS
# ============================================================

# ============================================================
# CORE NUMERICAL & DATA HANDLING
# ============================================================
import numpy as np
import pandas as pd

# ============================================================
# VISUALIZATION
# ============================================================
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================
# STATISTICAL TESTING
# ============================================================
from scipy import stats

# ============================================================
# SKLEARN: MODEL SELECTION & VALIDATION
# ============================================================
from sklearn.model_selection import (
    StratifiedKFold,
    train_test_split
)

# ============================================================
# SKLEARN: MODELS
# ============================================================
from sklearn.ensemble import RandomForestClassifier

# ============================================================
# SKLEARN: METRICS & FEATURE IMPORTANCE
# ============================================================
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    mutual_info_score
)
from sklearn.inspection import permutation_importance

# ============================================================
# GRADIENT BOOSTING
# ============================================================
import xgboost as xgb

# ============================================================
# RULE / ILP / STRUCTURAL UTILITIES
# ============================================================
import itertools
from collections import defaultdict
from dataclasses import dataclass
from typing import List, Dict, Tuple, Set

# ============================================================
# WARNINGS CONTROL
# ============================================================
import warnings
warnings.filterwarnings("ignore")

In [3]:
# ============================================================
# CELL 2: LOAD & INSPECT DATA
# ============================================================

# Load dataset
path = "/content/sample_data/bank-full.csv"
df = pd.read_csv(path, sep=";")

print(f"Shape: {df.shape}")
df.head()

Shape: (45211, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
# ============================================================
# CELL 3: CHECK INPUT TYPES
# ============================================================
print("📌 COLUMN SCHEMA\n" + "-"*60)
schema = pd.DataFrame({
    "column": df.columns,
    "dtype": df.dtypes.astype(str),
    "n_unique": [df[c].nunique() for c in df.columns]
})
schema

📌 COLUMN SCHEMA
------------------------------------------------------------


,column,dtype,n_unique
age,age,int64,77
job,job,object,12
marital,marital,object,3
education,education,object,4
default,default,object,2
balance,balance,int64,7168
housing,housing,object,2
loan,loan,object,2
contact,contact,object,3
day,day,int64,31


In [8]:
# ============================================================
# CELL 4: EXAMINE CATAGORICAL COLUMNS
# ============================================================

categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()

print("📌 CATEGORICAL COLUMNS\n" + "-"*60)
categorical_cols

for col in categorical_cols:
    print(f"\n🔎 Column: {col}")
    print("-" * 40)

    values = (
        df[col]
        .astype(str)
        .value_counts(dropna=False)
        .sort_index()
    )

    print(values)

📌 CATEGORICAL COLUMNS
------------------------------------------------------------

🔎 Column: job
----------------------------------------
job
admin.           5171
blue-collar      9732
entrepreneur     1487
housemaid        1240
management       9458
retired          2264
self-employed    1579
services         4154
student           938
technician       7597
unemployed       1303
unknown           288
Name: count, dtype: int64

🔎 Column: marital
----------------------------------------
marital
divorced     5207
married     27214
single      12790
Name: count, dtype: int64

🔎 Column: education
----------------------------------------
education
primary       6851
secondary    23202
tertiary     13301
unknown       1857
Name: count, dtype: int64

🔎 Column: default
----------------------------------------
default
no     44396
yes      815
Name: count, dtype: int64

🔎 Column: housing
----------------------------------------
housing
no     20081
yes    25130
Name: count, dtype: int64

🔎 Co

In [9]:
# ============================================================
# CELL 5: PREPROCESSING
# ============================================================

print("\n" + "="*80)
print("🔧 CLEAN PREPROCESSING")
print("="*80)

print(f"\nOriginal data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# --------------------------------------------------
# 1. CREATE CLEAN df_proc FRAME
# --------------------------------------------------
print("\n" + "="*40)
print("1. CREATING CLEAN df_proc")
print("="*40)

df_proc = df.copy()

print(f"\nInitial shape: {df_proc.shape}")
print(f"Columns: {list(df_proc.columns)}")

# --------------------------------------------------
# 1.1 Target encoding
# --------------------------------------------------
print(f"\n🔍 Target column 'y' before conversion:")
print(df_proc['y'].value_counts())

df_proc['y'] = df_proc['y'].map({'yes': 1, 'no': 0})
assert df_proc['y'].notna().all(), "❌ Unexpected values in target y"
df_proc['y'] = df_proc['y'].astype('int8')

print(f"\n✅ Target 'y' converted:")
print(df_proc['y'].value_counts())

# --------------------------------------------------
# 1.2 Binary categorical encoding
# --------------------------------------------------
binary_cols = ['default', 'housing', 'loan']
binary_map = {'no': 0, 'yes': 1}

print(f"\n🔍 Binary columns before conversion:")
for col in binary_cols:
    print(f"{col}:\n{df_proc[col].value_counts()}")

for col in binary_cols:
    df_proc[col] = df_proc[col].map(binary_map)
    assert df_proc[col].notna().all(), f"❌ Unexpected values in {col}"
    df_proc[col] = df_proc[col].astype('int8')

print(f"\n✅ Binary columns converted:")
for col in binary_cols:
    print(f"{col}: {df_proc[col].unique()}")

# --------------------------------------------------
# 1.3 Month → ordinal (DROP RAW MONTH)
# --------------------------------------------------
print(f"\n🔍 Month column before conversion:")
print(df_proc['month'].value_counts().sort_index())

month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

df_proc['month_ordinal'] = df_proc['month'].map(month_map)
assert df_proc['month_ordinal'].notna().all(), "❌ Unexpected month values"
df_proc['month_ordinal'] = df_proc['month_ordinal'].astype('int8')

# DROP raw month (no objects allowed)
df_proc.drop(columns=['month'], inplace=True)

print(f"\n✅ Month converted to ordinal and raw month dropped")

# --------------------------------------------------
# 1.4 POUTCOME - EXPLICIT MAPPING
# --------------------------------------------------
print(f"\n🔍 poutcome column before conversion:")
print(df_proc['poutcome'].value_counts())

poutcome_map = {
    'unknown': 0,
    'failure': 1,
    'other': 2,
    'success': 3
}

df_proc['poutcome'] = df_proc['poutcome'].map(poutcome_map)
assert df_proc['poutcome'].notna().all(), "❌ Unexpected poutcome values"
df_proc['poutcome'] = df_proc['poutcome'].astype('int8')

print(f"\n✅ poutcome converted:")
print(df_proc['poutcome'].value_counts())

# --------------------------------------------------
# 1.5 CONTACT - ORDINAL MAPPING
# --------------------------------------------------
print(f"\n🔍 contact column before conversion:")
print(df_proc['contact'].value_counts())

# Map contact as ORDINAL: cellular=0, telephone=1, unknown=2
contact_map = {
    'cellular': 0,
    'telephone': 1,
    'unknown': 2
}

df_proc['contact'] = df_proc['contact'].map(contact_map)
assert df_proc['contact'].notna().all(), "❌ Unexpected contact values"
df_proc['contact'] = df_proc['contact'].astype('int8')

print(f"\n✅ contact converted to ordinal (0=cellular, 1=telephone, 2=unknown):")
print(df_proc['contact'].value_counts().sort_index())

# --------------------------------------------------
# 1.6 Ordinal / Label encoding REMAINING categoricals
# --------------------------------------------------
categorical_cols = df_proc.select_dtypes(include=['object']).columns.tolist()

print(f"\n🔍 Ordinal-encoding remaining categoricals:")
print(categorical_cols)

for col in categorical_cols:
    df_proc[col] = df_proc[col].astype('category').cat.codes
    df_proc[col] = df_proc[col].astype('int16')

# --------------------------------------------------
# 1.7 HARD TYPE GUARANTEE
# --------------------------------------------------
non_numeric = df_proc.select_dtypes(exclude=['number']).columns.tolist()
assert len(non_numeric) == 0, f"❌ NON-NUMERIC COLUMNS REMAIN: {non_numeric}"

print("\n✅ ALL COLUMNS ARE NUMERIC")

# --------------------------------------------------
# 1.8 Sanity checks
# --------------------------------------------------
print(f"\n📊 Data types in df_proc:")
for dtype in df_proc.dtypes.unique():
    cols = df_proc.columns[df_proc.dtypes == dtype].tolist()
    print(f"{dtype}: {cols}")

print(f"\n🔍 Checking for null values:")
assert df_proc.isnull().sum().sum() == 0, "❌ Nulls detected"
print("✅ No null values")

print(f"\n🔍 Checking for duplicates:")
dup_count = df_proc.duplicated().sum()
print(f"Exact duplicate rows: {dup_count} ({dup_count/len(df_proc)*100:.2f}%)")

# --------------------------------------------------
# 1.9 Final confirmation
# --------------------------------------------------
print(f"\n✅ df_proc created successfully!")
print(f"Shape: {df_proc.shape}")
print(f"Memory usage: {df_proc.memory_usage().sum() / 1024**2:.2f} MB")


🔧 CLEAN PREPROCESSING

Original data shape: (45211, 17)
Columns: ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

1. CREATING CLEAN df_proc

Initial shape: (45211, 17)
Columns: ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

🔍 Target column 'y' before conversion:
y
no     39922
yes     5289
Name: count, dtype: int64

✅ Target 'y' converted:
y
0    39922
1     5289
Name: count, dtype: int64

🔍 Binary columns before conversion:
default:
default
no     44396
yes      815
Name: count, dtype: int64
housing:
housing
yes    25130
no     20081
Name: count, dtype: int64
loan:
loan
no     37967
yes     7244
Name: count, dtype: int64

✅ Binary columns converted:
default: [0 1]
housing: [1 0]
loan: [0 1]

🔍 Month column before conversion:
month
apr     2932


In [7]:
!pip install interpret -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 83.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.1/780.1 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 11.9 MB/s eta 0:00:00


In [17]:
# ============================================================
# CELL: META-EBM (STAGE 4) - COMPLETE IMPLEMENTATION
# Includes all necessary class definitions for unpickling
# ============================================================

print("\n" + "="*80)
print("🎯 META-EBM (STAGE 4) - FINAL CASCADE ASSEMBLY")
print("="*80)

import pickle
import joblib
import pandas as pd
import numpy as np
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix,
    classification_report
)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from dataclasses import dataclass
from typing import Set, Tuple, List, Dict
from collections import defaultdict

# --------------------------------------------------
# 1. LOAD ALL STAGE MODELS
# --------------------------------------------------
print("\n" + "="*60)
print("📂 1. LOADING STAGE 1-3 MODELS")
print("="*60)

# Stage 1: Logistic Regression (load from dict)
print("\n🔹 Loading Logistic Regression (Stage 1)...")
lr_ensemble_dict = joblib.load('/content/lr_ensemble_joblib_20251220_203823.joblib')
lr_model = lr_ensemble_dict['model']
print(f"✅ LR model loaded")

# Stage 2: GLASS-BRW
print("\n🔹 Loading GLASS-BRW (Stage 2)...")
with open('/content/glass_brw_rf_checkpoint_20251220_211009.pkl', 'rb') as f:
    glass_model = pickle.load(f)
print(f"✅ GLASS-BRW model loaded - {len(glass_model.rules)} rules")

# Extract RF model
rf_model = glass_model.rf_model
print(f"✅ RF model extracted")

# Stage 3: EBM (load from dict)
print("\n🔹 Loading EBM (Stage 3)...")
ebm_ensemble_dict = joblib.load('/content/ebm_ensemble_joblib_20251220_223007.joblib')
ebm_model = ebm_ensemble_dict['model']
print(f"✅ EBM model loaded")

# --------------------------------------------------
# 2. PREPARE DATA - USE PREDICTIONS FROM SAVED MODELS
# --------------------------------------------------
print("\n" + "="*60)
print("🔧 2. EXTRACTING PREDICTIONS FROM TRAINED MODELS")
print("="*60)

# The models were already trained on specific train/test splits
# We need to use THEIR predictions, not generate new ones

# Stage 1: LR predictions (already computed during training)
lr_X_train = lr_ensemble_dict['train_predictions']
lr_X_test = lr_ensemble_dict['test_predictions']
y_train = lr_ensemble_dict['train_labels']
y_test = lr_ensemble_dict['test_labels']

print(f"✅ LR predictions extracted from saved model:")
print(f"  Train: {lr_X_train.shape} - mean: {lr_X_train.mean():.4f}")
print(f"  Test:  {lr_X_test.shape} - mean: {lr_X_test.mean():.4f}")

# Stage 3: EBM predictions (already computed during training)
ebm_X_train = ebm_ensemble_dict['train_predictions']
ebm_X_test = ebm_ensemble_dict['test_predictions']

print(f"✅ EBM predictions extracted from saved model:")
print(f"  Train: {ebm_X_train.shape} - mean: {ebm_X_train.mean():.4f}")
print(f"  Test:  {ebm_X_test.shape} - mean: {ebm_X_test.mean():.4f}")

# Verify shapes match
assert len(lr_X_train) == len(ebm_X_train), "Train set size mismatch!"
assert len(lr_X_test) == len(ebm_X_test), "Test set size mismatch!"

print(f"\n✅ Consistent split verified:")
print(f"  Train samples: {len(lr_X_train)}")
print(f"  Test samples: {len(lr_X_test)}")

# --------------------------------------------------
# 3. GENERATE GLASS-BRW PREDICTIONS ON SAME SPLIT
# --------------------------------------------------
print("\n" + "="*60)
print("🎲 3. GENERATING GLASS-BRW PREDICTIONS")
print("="*60)

# We need to regenerate GLASS predictions on the SAME split
# First, get the original data and recreate the exact split
X_all = df_proc.drop(columns=['y'])
y_all = df_proc['y']

# Use the same split as LR/EBM (they used random_state=42, test_size=0.2)
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(
    X_all, y_all, test_size=0.2, random_state=42, stratify=y_all
)

# Verify this matches our saved predictions
assert len(X_train_orig) == len(lr_X_train), "Train split doesn't match!"
assert len(X_test_orig) == len(lr_X_test), "Test split doesn't match!"

print(f"✅ Recreated original train/test split")
print(f"  Train: {X_train_orig.shape}")
print(f"  Test:  {X_test_orig.shape}")

# --------------------------------------------------
# 4. ENGINEER FEATURES FOR GLASS-BRW
# --------------------------------------------------
print("\n" + "="*60)
print("🔨 4. ENGINEERING FEATURES FOR GLASS-BRW")
print("="*60)

def engineer_features_bank(df_proc: pd.DataFrame) -> pd.DataFrame:
    df = df_proc.copy()
    df["dur_low"]  = (df["duration"] <= 200).astype("int8")
    df["dur_mid"]  = ((df["duration"] > 200) & (df["duration"] <= 300)).astype("int8")
    df["dur_high"] = (df["duration"] > 300).astype("int8")
    df["pdays_never"] = (df["pdays"] <= 0).astype("int8")
    df["pdays_1_5"]   = ((df["pdays"] > 0) & (df["pdays"] <= 5)).astype("int8")
    df["month_early"]      = df["month_ordinal"].isin([1, 2, 3, 4]).astype("int8")
    df["month_late_spring"] = df["month_ordinal"].isin([5, 6]).astype("int8")
    df["month_summer"]     = df["month_ordinal"].isin([7, 8]).astype("int8")
    df["month_fall"]       = df["month_ordinal"].isin([9, 10]).astype("int8")
    df["month_year_end"]   = df["month_ordinal"].isin([11, 12]).astype("int8")
    df["previous_ge1"] = (df["previous"] >= 1).astype("int8")
    df["engagement_score"] = (
        df["previous_ge1"] + df["pdays_1_5"] + df["dur_high"] + (df["poutcome"] == 3).astype("int8")
    ).astype("int8")
    df["fatigue_score"] = (
        (df["campaign"] >= 5).astype("int8") + (df["campaign"] >= 8).astype("int8") +
        df["dur_low"] + df["pdays_never"]
    ).astype("int8")

    df_eng = pd.DataFrame(index=df.index)
    df_eng["dur_low"]  = df["dur_low"]
    df_eng["dur_mid"]  = df["dur_mid"]
    df_eng["dur_high"] = df["dur_high"]
    df_eng["month_summer"]   = df["month_summer"]
    df_eng["month_fall"]     = df["month_fall"]
    df_eng["month_year_end"] = df["month_year_end"]
    df_eng["month_early"] = df["month_early"]
    df_eng["month_late_spring"] = df["month_late_spring"]
    df_eng["engagement_low"] = (df["engagement_score"] <= 1).astype("int8")
    df_eng["fatigue_low"]    = (df["fatigue_score"] <= 1).astype("int8")
    df_eng["housing"]          = df["housing"].astype("int8")
    df_eng["poutcome_success"] = (df["poutcome"] == 3).astype("int8")
    df_eng["pdays_never"]      = df["pdays_never"]
    return df_eng

X_train_glass = engineer_features_bank(X_train_orig.copy())
X_test_glass = engineer_features_bank(X_test_orig.copy())

glass_X_train = glass_model.predict_proba(X_train_glass)[:, 1]
glass_X_test = glass_model.predict_proba(X_test_glass)[:, 1]

print(f"✅ GLASS predictions generated:")
print(f"  Train: {glass_X_train.shape} - mean: {glass_X_train.mean():.4f}")
print(f"  Test:  {glass_X_test.shape} - mean: {glass_X_test.mean():.4f}")

# --------------------------------------------------
# 5. CREATE META-FEATURES (NO DATA LEAKAGE)
# --------------------------------------------------
print("\n" + "="*60)
print("🔨 5. CREATING META-FEATURES")
print("="*60)

# Verify all predictions are on the same samples
assert len(lr_X_train) == len(glass_X_train) == len(ebm_X_train), "Train size mismatch!"
assert len(lr_X_test) == len(glass_X_test) == len(ebm_X_test), "Test size mismatch!"

print(f"✅ All models have predictions on same split:")
print(f"  Train: {len(lr_X_train)} samples")
print(f"  Test:  {len(lr_X_test)} samples")

print(f"\n📊 Stage prediction means:")
print(f"  LR:    train={lr_X_train.mean():.4f}, test={lr_X_test.mean():.4f}")
print(f"  GLASS: train={glass_X_train.mean():.4f}, test={glass_X_test.mean():.4f}")
print(f"  EBM:   train={ebm_X_train.mean():.4f}, test={ebm_X_test.mean():.4f}")

meta_train = pd.DataFrame({
    'lr_prob': lr_X_train,
    'rf_prob': glass_X_train,
    'ebm_prob': ebm_X_train
})

meta_test = pd.DataFrame({
    'lr_prob': lr_X_test,
    'rf_prob': glass_X_test,
    'ebm_prob': ebm_X_test
})

# Disagreement signals
meta_train['lr_rf_diff'] = np.abs(meta_train['lr_prob'] - meta_train['rf_prob'])
meta_train['lr_ebm_diff'] = np.abs(meta_train['lr_prob'] - meta_train['ebm_prob'])
meta_train['rf_ebm_diff'] = np.abs(meta_train['rf_prob'] - meta_train['ebm_prob'])

meta_test['lr_rf_diff'] = np.abs(meta_test['lr_prob'] - meta_test['rf_prob'])
meta_test['lr_ebm_diff'] = np.abs(meta_test['lr_prob'] - meta_test['ebm_prob'])
meta_test['rf_ebm_diff'] = np.abs(meta_test['rf_prob'] - meta_test['ebm_prob'])

# Confidence geometry
P_train = meta_train[['lr_prob', 'rf_prob', 'ebm_prob']].values
meta_train['max_confidence'] = P_train.max(axis=1)
meta_train['min_confidence'] = P_train.min(axis=1)
meta_train['std_confidence'] = P_train.std(axis=1)

P_test = meta_test[['lr_prob', 'rf_prob', 'ebm_prob']].values
meta_test['max_confidence'] = P_test.max(axis=1)
meta_test['min_confidence'] = P_test.min(axis=1)
meta_test['std_confidence'] = P_test.std(axis=1)

print(f"✅ Meta-features: {meta_train.shape}")
print(f"   Features: {list(meta_train.columns)}")

# --------------------------------------------------
# 6. TRAIN META-EBM
# --------------------------------------------------
print("\n" + "="*60)
print("🎯 6. TRAINING META-EBM (10-FOLD CV)")
print("="*60)

meta_ebm = ExplainableBoostingClassifier(
    max_bins=16,
    max_interaction_bins=8,
    interactions=5,
    outer_bags=10,
    inner_bags=0,
    learning_rate=0.01,
    min_samples_leaf=5,
    max_leaves=3,
    random_state=42,
    n_jobs=-1
)

meta_ebm.fit(meta_train, y_train)
print("✅ Meta-EBM trained!")

# --------------------------------------------------
# 7. EVALUATE
# --------------------------------------------------
print("\n" + "="*60)
print("📊 7. EVALUATION")
print("="*60)

meta_pred_test = meta_ebm.predict_proba(meta_test)[:, 1]
meta_pred_test_class = (meta_pred_test >= 0.5).astype(int)

accuracy = accuracy_score(y_test, meta_pred_test_class)
precision = precision_score(y_test, meta_pred_test_class)
recall = recall_score(y_test, meta_pred_test_class)
f1 = f1_score(y_test, meta_pred_test_class)
roc_auc = roc_auc_score(y_test, meta_pred_test)

print(f"\n🎯 META-EBM PERFORMANCE:")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")
print(f"  ROC-AUC:   {roc_auc:.4f}")

cm = confusion_matrix(y_test, meta_pred_test_class)
print(f"\nConfusion Matrix:\n{cm}")

# --------------------------------------------------
# 9. SAVE
# --------------------------------------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint = {
    'meta_ebm_model': meta_ebm,
    'performance': {'accuracy': accuracy, 'precision': precision, 'recall': recall,
                   'f1_score': f1, 'roc_auc': roc_auc},
    'training_date': datetime.now().isoformat()
}

filename = f"/content/meta_ebm_cascade_checkpoint_{timestamp}.pkl"
with open(filename, 'wb') as f:
    pickle.dump(checkpoint, f)

print(f"\n✅ Saved: {filename}")
print("\n" + "="*80)
print("🎉 4-STAGE CASCADE COMPLETE!")
print("="*80)


🎯 META-EBM (STAGE 4) - FINAL CASCADE ASSEMBLY

📂 1. LOADING STAGE 1-3 MODELS

🔹 Loading Logistic Regression (Stage 1)...
✅ LR model loaded

🔹 Loading GLASS-BRW (Stage 2)...
✅ GLASS-BRW model loaded - 7 rules
✅ RF model extracted

🔹 Loading EBM (Stage 3)...
✅ EBM model loaded

🔧 2. EXTRACTING PREDICTIONS FROM TRAINED MODELS
✅ LR predictions extracted from saved model:
  Train: (36168,) - mean: 0.3306
  Test:  (9043,) - mean: 0.3316
✅ EBM predictions extracted from saved model:
  Train: (36168,) - mean: 0.1170
  Test:  (9043,) - mean: 0.1157

✅ Consistent split verified:
  Train samples: 36168
  Test samples: 9043

🎲 3. GENERATING GLASS-BRW PREDICTIONS
✅ Recreated original train/test split
  Train: (36168, 16)
  Test:  (9043, 16)

🔨 4. ENGINEERING FEATURES FOR GLASS-BRW
✅ GLASS predictions generated:
  Train: (36168,) - mean: 0.0993
  Test:  (9043,) - mean: 0.1002

🔨 5. CREATING META-FEATURES
✅ All models have predictions on same split:
  Train: 36168 samples
  Test:  9043 samples

📊 St

In [18]:
# ============================================================
# CELL: META-EBM (STAGE 4) - PROPER DATA SPLIT
# ============================================================

print("\n" + "="*80)
print("🎯 META-EBM (STAGE 4) - FINAL CASCADE ASSEMBLY")
print("="*80)

import pickle
import joblib
import pandas as pd
import numpy as np
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix,
    classification_report
)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from datetime import datetime
from dataclasses import dataclass
from typing import Set, Tuple, List, Dict
from collections import defaultdict

# Execute class definitions for unpickling
exec("""
@dataclass
class Rule:
    rule_id: int
    segment: Set[Tuple[str, str]]
    predicted_class: int
    complexity: int
    precision: float = 0.0
    recall: float = 0.0
    coverage: float = 0.0
    stability: float = 0.0
    interpretability: float = 0.0
    boundary_ambiguity: float = 0.0
    rf_alignment: float = 0.0
    def __hash__(self):
        return hash((self.rule_id, frozenset(self.segment), self.predicted_class))
    def __eq__(self, other):
        return self.rule_id == other.rule_id

class BankSegmentBuilder:
    SEGMENT_FEATURES = [
        "dur_low", "dur_mid", "dur_high",
        "month_early", "month_late_spring", "month_summer", "month_fall", "month_year_end",
        "engagement_low", "fatigue_low", "poutcome_success",
        "housing", "pdays_never"
    ]
    def assign_segments(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.drop(columns=["y", "y_bin"], errors="ignore")
        cols = [c for c in self.SEGMENT_FEATURES if c in X.columns]
        if len(cols) == 0:
            raise ValueError("No valid segment features found in input")
        segments = X[cols].copy()
        for col in segments.columns:
            vals = set(segments[col].dropna().unique())
            if not vals.issubset({0, 1}):
                raise ValueError(f"Non-binary segment feature: {col} -> {vals}")
        return segments.astype("int8")

class RuleGenerator:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class RuleEvaluator:
    def __init__(self, segment_builder, min_support=30):
        self.segment_builder = segment_builder
        self.min_support = min_support

class ILPRuleSelector:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class GLASS_BRW:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)
        if not hasattr(self, 'segment_builder'):
            self.segment_builder = BankSegmentBuilder()

    def predict_proba(self, X):
        if not hasattr(self, 'rules') or not self.rules:
            n = len(X)
            return np.column_stack([np.full(n, 0.5), np.full(n, 0.5)])

        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        n = len(X)
        predictions = np.full(n, -1, dtype=int)
        confidences = np.zeros(n)

        segments = self.segment_builder.assign_segments(X)

        for i in range(n):
            for rule in self.rules:
                matches = True
                for feature, level in rule.segment:
                    if feature not in segments.columns:
                        matches = False
                        break
                    if segments.iloc[i][feature] != level:
                        matches = False
                        break

                if not matches:
                    continue

                predictions[i] = rule.predicted_class
                confidences[i] = rule.precision
                break

        probas = np.zeros((n, 2))
        for i in range(n):
            if predictions[i] == -1:
                probas[i] = [0.5, 0.5]
            elif predictions[i] == 1:
                probas[i] = [1 - confidences[i], confidences[i]]
            else:
                probas[i] = [confidences[i], 1 - confidences[i]]

        return probas
""")

print("✅ Class definitions loaded")

# --------------------------------------------------
# 1. LOAD ALL MODELS
# --------------------------------------------------
print("\n" + "="*60)
print("📂 1. LOADING STAGE 1-3 MODELS")
print("="*60)

# Load LR
lr_ensemble_dict = joblib.load('/content/lr_ensemble_joblib_20251220_203823.joblib')
lr_model = lr_ensemble_dict['model']
scaler = lr_ensemble_dict['scaler']
print(f"✅ LR model loaded")

# Load GLASS-BRW
with open('/content/glass_brw_rf_checkpoint_20251220_211009.pkl', 'rb') as f:
    glass_model = pickle.load(f)
rf_model = glass_model.rf_model
print(f"✅ GLASS-BRW model loaded ({len(glass_model.rules)} rules)")

# Load EBM
ebm_ensemble_dict = joblib.load('/content/ebm_ensemble_joblib_20251220_223007.joblib')
ebm_model = ebm_ensemble_dict['model']
print(f"✅ EBM model loaded")

# --------------------------------------------------
# 2. CREATE SINGLE TRAIN/TEST SPLIT FOR ALL MODELS
# --------------------------------------------------
print("\n" + "="*60)
print("🔧 2. CREATING SINGLE TRAIN/TEST SPLIT")
print("="*60)

# Split the full dataset
X_full = df_proc.drop(columns=['y'])
y_full = df_proc['y']

X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42, stratify=y_full
)

print(f"✅ Single split created:")
print(f"  Train: {X_train.shape} samples")
print(f"  Test:  {X_test.shape} samples")
print(f"  Train positives: {y_train.mean():.4f}")
print(f"  Test positives:  {y_test.mean():.4f}")

# --------------------------------------------------
# 3. GENERATE PREDICTIONS FROM ALL STAGES ON SAME SPLIT
# --------------------------------------------------
print("\n" + "="*60)
print("🎲 3. GENERATING PREDICTIONS FROM ALL STAGES")
print("="*60)

# Stage 1: Logistic Regression
print("\n🔹 Stage 1: Logistic Regression")
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr_X_train = lr_model.predict_proba(X_train_scaled)[:, 1]
lr_X_test = lr_model.predict_proba(X_test_scaled)[:, 1]

print(f"  Train: mean={lr_X_train.mean():.4f}, std={lr_X_train.std():.4f}")
print(f"  Test:  mean={lr_X_test.mean():.4f}, std={lr_X_test.std():.4f}")

# Stage 2: GLASS-BRW (needs feature engineering)
print("\n🔹 Stage 2: GLASS-BRW")

def engineer_features_bank(df: pd.DataFrame) -> pd.DataFrame:
    """Engineer features for GLASS-BRW"""
    d = df.copy()
    d["dur_low"] = (d["duration"] <= 200).astype("int8")
    d["dur_mid"] = ((d["duration"] > 200) & (d["duration"] <= 300)).astype("int8")
    d["dur_high"] = (d["duration"] > 300).astype("int8")
    d["pdays_never"] = (d["pdays"] <= 0).astype("int8")
    d["pdays_1_5"] = ((d["pdays"] > 0) & (d["pdays"] <= 5)).astype("int8")
    d["month_early"] = d["month_ordinal"].isin([1, 2, 3, 4]).astype("int8")
    d["month_late_spring"] = d["month_ordinal"].isin([5, 6]).astype("int8")
    d["month_summer"] = d["month_ordinal"].isin([7, 8]).astype("int8")
    d["month_fall"] = d["month_ordinal"].isin([9, 10]).astype("int8")
    d["month_year_end"] = d["month_ordinal"].isin([11, 12]).astype("int8")
    d["previous_ge1"] = (d["previous"] >= 1).astype("int8")
    d["engagement_score"] = (
        d["previous_ge1"] + d["pdays_1_5"] + d["dur_high"] + (d["poutcome"] == 3).astype("int8")
    ).astype("int8")
    d["fatigue_score"] = (
        (d["campaign"] >= 5).astype("int8") + (d["campaign"] >= 8).astype("int8") +
        d["dur_low"] + d["pdays_never"]
    ).astype("int8")

    out = pd.DataFrame(index=d.index)
    out["dur_low"] = d["dur_low"]
    out["dur_mid"] = d["dur_mid"]
    out["dur_high"] = d["dur_high"]
    out["month_summer"] = d["month_summer"]
    out["month_fall"] = d["month_fall"]
    out["month_year_end"] = d["month_year_end"]
    out["month_early"] = d["month_early"]
    out["month_late_spring"] = d["month_late_spring"]
    out["engagement_low"] = (d["engagement_score"] <= 1).astype("int8")
    out["fatigue_low"] = (d["fatigue_score"] <= 1).astype("int8")
    out["housing"] = d["housing"].astype("int8")
    out["poutcome_success"] = (d["poutcome"] == 3).astype("int8")
    out["pdays_never"] = d["pdays_never"]
    return out

X_train_glass = engineer_features_bank(X_train)
X_test_glass = engineer_features_bank(X_test)

glass_X_train = glass_model.predict_proba(X_train_glass)[:, 1]
glass_X_test = glass_model.predict_proba(X_test_glass)[:, 1]

print(f"  Train: mean={glass_X_train.mean():.4f}, std={glass_X_train.std():.4f}")
print(f"  Test:  mean={glass_X_test.mean():.4f}, std={glass_X_test.std():.4f}")

# Stage 3: EBM
print("\n🔹 Stage 3: EBM")
ebm_X_train = ebm_model.predict_proba(X_train)[:, 1]
ebm_X_test = ebm_model.predict_proba(X_test)[:, 1]

print(f"  Train: mean={ebm_X_train.mean():.4f}, std={ebm_X_train.std():.4f}")
print(f"  Test:  mean={ebm_X_test.mean():.4f}, std={ebm_X_test.std():.4f}")

# Verify all predictions are on same samples
assert len(lr_X_train) == len(glass_X_train) == len(ebm_X_train)
assert len(lr_X_test) == len(glass_X_test) == len(ebm_X_test)
print(f"\n✅ All models have predictions on same {len(lr_X_train)} train / {len(lr_X_test)} test samples")

# --------------------------------------------------
# 4. CREATE META-FEATURES
# --------------------------------------------------
print("\n" + "="*60)
print("🔨 4. CREATING META-FEATURES")
print("="*60)

meta_train = pd.DataFrame({
    'lr_prob': lr_X_train,
    'rf_prob': glass_X_train,
    'ebm_prob': ebm_X_train
})

meta_test = pd.DataFrame({
    'lr_prob': lr_X_test,
    'rf_prob': glass_X_test,
    'ebm_prob': ebm_X_test
})

# Disagreement signals
meta_train['lr_rf_diff'] = np.abs(meta_train['lr_prob'] - meta_train['rf_prob'])
meta_train['lr_ebm_diff'] = np.abs(meta_train['lr_prob'] - meta_train['ebm_prob'])
meta_train['rf_ebm_diff'] = np.abs(meta_train['rf_prob'] - meta_train['ebm_prob'])

meta_test['lr_rf_diff'] = np.abs(meta_test['lr_prob'] - meta_test['rf_prob'])
meta_test['lr_ebm_diff'] = np.abs(meta_test['lr_prob'] - meta_test['ebm_prob'])
meta_test['rf_ebm_diff'] = np.abs(meta_test['rf_prob'] - meta_test['ebm_prob'])

# Confidence geometry
P_train = meta_train[['lr_prob', 'rf_prob', 'ebm_prob']].values
meta_train['max_confidence'] = P_train.max(axis=1)
meta_train['min_confidence'] = P_train.min(axis=1)
meta_train['std_confidence'] = P_train.std(axis=1)

P_test = meta_test[['lr_prob', 'rf_prob', 'ebm_prob']].values
meta_test['max_confidence'] = P_test.max(axis=1)
meta_test['min_confidence'] = P_test.min(axis=1)
meta_test['std_confidence'] = P_test.std(axis=1)

print(f"✅ Meta-features created: {meta_train.shape}")
print(f"   Features: {list(meta_train.columns)}")

# --------------------------------------------------
# 5. TRAIN META-EBM (10-FOLD CV)
# --------------------------------------------------
print("\n" + "="*60)
print("🎯 5. TRAINING META-EBM (10-FOLD CV)")
print("="*60)

meta_ebm = ExplainableBoostingClassifier(
    max_bins=16,
    max_interaction_bins=8,
    interactions=5,
    outer_bags=10,
    inner_bags=0,
    learning_rate=0.01,
    min_samples_leaf=5,
    max_leaves=3,
    random_state=42,
    n_jobs=-1
)

meta_ebm.fit(meta_train, y_train)
print("✅ Meta-EBM trained!")

# --------------------------------------------------
# 6. EVALUATE
# --------------------------------------------------
print("\n" + "="*60)
print("📊 6. META-EBM EVALUATION")
print("="*60)

meta_pred_test = meta_ebm.predict_proba(meta_test)[:, 1]
meta_pred_test_class = (meta_pred_test >= 0.5).astype(int)

accuracy = accuracy_score(y_test, meta_pred_test_class)
precision = precision_score(y_test, meta_pred_test_class)
recall = recall_score(y_test, meta_pred_test_class)
f1 = f1_score(y_test, meta_pred_test_class)
roc_auc = roc_auc_score(y_test, meta_pred_test)

print(f"\n🎯 META-EBM PERFORMANCE:")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")
print(f"  ROC-AUC:   {roc_auc:.4f}")

cm = confusion_matrix(y_test, meta_pred_test_class)
print(f"\nConfusion Matrix:\n{cm}")
print(f"\nClassification Report:")
print(classification_report(y_test, meta_pred_test_class,
                          target_names=['No Subscribe', 'Subscribe']))

# --------------------------------------------------
# 7. SAVE CHECKPOINT
# --------------------------------------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint = {
    'meta_ebm_model': meta_ebm,
    'stage_models': {
        'lr_model': lr_model,
        'glass_model': glass_model,
        'rf_model': rf_model,
        'ebm_model': ebm_model
    },
    'performance': {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc
    },
    'training_date': datetime.now().isoformat()
}

filename = f"/content/meta_ebm_cascade_checkpoint_{timestamp}.pkl"
with open(filename, 'wb') as f:
    pickle.dump(checkpoint, f)

print(f"\n✅ Saved: {filename}")
print("\n" + "="*80)
print("🎉 4-STAGE CASCADE COMPLETE!")
print("="*80)


🎯 META-EBM (STAGE 4) - FINAL CASCADE ASSEMBLY
✅ Class definitions loaded

📂 1. LOADING STAGE 1-3 MODELS
✅ LR model loaded
✅ GLASS-BRW model loaded (7 rules)
✅ EBM model loaded

🔧 2. CREATING SINGLE TRAIN/TEST SPLIT
✅ Single split created:
  Train: (36168, 16) samples
  Test:  (9043, 16) samples
  Train positives: 0.1170
  Test positives:  0.1170

🎲 3. GENERATING PREDICTIONS FROM ALL STAGES

🔹 Stage 1: Logistic Regression
  Train: mean=0.3306, std=0.2785
  Test:  mean=0.3316, std=0.2785

🔹 Stage 2: GLASS-BRW
  Train: mean=0.0993, std=0.1171
  Test:  mean=0.1002, std=0.1191

🔹 Stage 3: EBM
  Train: mean=0.1170, std=0.2092
  Test:  mean=0.1157, std=0.2064

✅ All models have predictions on same 36168 train / 9043 test samples

🔨 4. CREATING META-FEATURES
✅ Meta-features created: (36168, 9)
   Features: ['lr_prob', 'rf_prob', 'ebm_prob', 'lr_rf_diff', 'lr_ebm_diff', 'rf_ebm_diff', 'max_confidence', 'min_confidence', 'std_confidence']

🎯 5. TRAINING META-EBM (10-FOLD CV)
✅ Meta-EBM trained!

In [19]:
# ============================================================
# CELL: META-EBM (STAGE 4) - PROPER DATA SPLIT
# ============================================================

print("\n" + "="*80)
print("🎯 META-EBM (STAGE 4) - FINAL CASCADE ASSEMBLY")
print("="*80)

import pickle
import joblib
import pandas as pd
import numpy as np
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix,
    classification_report
)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from datetime import datetime
from dataclasses import dataclass
from typing import Set, Tuple, List, Dict
from collections import defaultdict

# Execute class definitions for unpickling
exec("""
@dataclass
class Rule:
    rule_id: int
    segment: Set[Tuple[str, str]]
    predicted_class: int
    complexity: int
    precision: float = 0.0
    recall: float = 0.0
    coverage: float = 0.0
    stability: float = 0.0
    interpretability: float = 0.0
    boundary_ambiguity: float = 0.0
    rf_alignment: float = 0.0
    def __hash__(self):
        return hash((self.rule_id, frozenset(self.segment), self.predicted_class))
    def __eq__(self, other):
        return self.rule_id == other.rule_id

class BankSegmentBuilder:
    SEGMENT_FEATURES = [
        "dur_low", "dur_mid", "dur_high",
        "month_early", "month_late_spring", "month_summer", "month_fall", "month_year_end",
        "engagement_low", "fatigue_low", "poutcome_success",
        "housing", "pdays_never"
    ]
    def assign_segments(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.drop(columns=["y", "y_bin"], errors="ignore")
        cols = [c for c in self.SEGMENT_FEATURES if c in X.columns]
        if len(cols) == 0:
            raise ValueError("No valid segment features found in input")
        segments = X[cols].copy()
        for col in segments.columns:
            vals = set(segments[col].dropna().unique())
            if not vals.issubset({0, 1}):
                raise ValueError(f"Non-binary segment feature: {col} -> {vals}")
        return segments.astype("int8")

class RuleGenerator:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class RuleEvaluator:
    def __init__(self, segment_builder, min_support=30):
        self.segment_builder = segment_builder
        self.min_support = min_support

class ILPRuleSelector:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class GLASS_BRW:
    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)
        if not hasattr(self, 'segment_builder'):
            self.segment_builder = BankSegmentBuilder()

    def predict_proba(self, X):
        if not hasattr(self, 'rules') or not self.rules:
            n = len(X)
            return np.column_stack([np.full(n, 0.5), np.full(n, 0.5)])

        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        n = len(X)
        predictions = np.full(n, -1, dtype=int)
        confidences = np.zeros(n)

        segments = self.segment_builder.assign_segments(X)

        for i in range(n):
            for rule in self.rules:
                matches = True
                for feature, level in rule.segment:
                    if feature not in segments.columns:
                        matches = False
                        break
                    if segments.iloc[i][feature] != level:
                        matches = False
                        break

                if not matches:
                    continue

                predictions[i] = rule.predicted_class
                confidences[i] = rule.precision
                break

        probas = np.zeros((n, 2))
        for i in range(n):
            if predictions[i] == -1:
                probas[i] = [0.5, 0.5]
            elif predictions[i] == 1:
                probas[i] = [1 - confidences[i], confidences[i]]
            else:
                probas[i] = [confidences[i], 1 - confidences[i]]

        return probas
""")

print("✅ Class definitions loaded")

# --------------------------------------------------
# 1. LOAD ALL MODELS
# --------------------------------------------------
print("\n" + "="*60)
print("📂 1. LOADING STAGE 1-3 MODELS")
print("="*60)

# Load LR
lr_ensemble_dict = joblib.load('/content/lr_ensemble_joblib_20251220_203823.joblib')
lr_model = lr_ensemble_dict['model']
scaler = lr_ensemble_dict['scaler']
print(f"✅ LR model loaded")

# Load GLASS-BRW
with open('/content/glass_brw_rf_checkpoint_20251220_211009.pkl', 'rb') as f:
    glass_model = pickle.load(f)
rf_model = glass_model.rf_model
print(f"✅ GLASS-BRW model loaded ({len(glass_model.rules)} rules)")

# Load EBM
ebm_ensemble_dict = joblib.load('/content/ebm_ensemble_joblib_20251220_223007.joblib')
ebm_model = ebm_ensemble_dict['model']
print(f"✅ EBM model loaded")

# --------------------------------------------------
# 2. CREATE SINGLE TRAIN/TEST SPLIT FOR ALL MODELS
# --------------------------------------------------
print("\n" + "="*60)
print("🔧 2. CREATING SINGLE TRAIN/TEST SPLIT")
print("="*60)

# Split the full dataset
X_full = df_proc.drop(columns=['y'])
y_full = df_proc['y']

X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42, stratify=y_full
)

print(f"✅ Single split created:")
print(f"  Train: {X_train.shape} samples")
print(f"  Test:  {X_test.shape} samples")
print(f"  Train positives: {y_train.mean():.4f}")
print(f"  Test positives:  {y_test.mean():.4f}")

# --------------------------------------------------
# 3. GENERATE PREDICTIONS FROM ALL STAGES ON SAME SPLIT
# --------------------------------------------------
print("\n" + "="*60)
print("🎲 3. GENERATING PREDICTIONS FROM ALL STAGES")
print("="*60)

# Stage 1: Logistic Regression
print("\n🔹 Stage 1: Logistic Regression")
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr_X_train = lr_model.predict_proba(X_train_scaled)[:, 1]
lr_X_test = lr_model.predict_proba(X_test_scaled)[:, 1]

print(f"  Train: mean={lr_X_train.mean():.4f}, std={lr_X_train.std():.4f}")
print(f"  Test:  mean={lr_X_test.mean():.4f}, std={lr_X_test.std():.4f}")

# Stage 2: GLASS-BRW (needs feature engineering)
print("\n🔹 Stage 2: GLASS-BRW")

def engineer_features_bank(df: pd.DataFrame) -> pd.DataFrame:
    """Engineer features for GLASS-BRW"""
    d = df.copy()
    d["dur_low"] = (d["duration"] <= 200).astype("int8")
    d["dur_mid"] = ((d["duration"] > 200) & (d["duration"] <= 300)).astype("int8")
    d["dur_high"] = (d["duration"] > 300).astype("int8")
    d["pdays_never"] = (d["pdays"] <= 0).astype("int8")
    d["pdays_1_5"] = ((d["pdays"] > 0) & (d["pdays"] <= 5)).astype("int8")
    d["month_early"] = d["month_ordinal"].isin([1, 2, 3, 4]).astype("int8")
    d["month_late_spring"] = d["month_ordinal"].isin([5, 6]).astype("int8")
    d["month_summer"] = d["month_ordinal"].isin([7, 8]).astype("int8")
    d["month_fall"] = d["month_ordinal"].isin([9, 10]).astype("int8")
    d["month_year_end"] = d["month_ordinal"].isin([11, 12]).astype("int8")
    d["previous_ge1"] = (d["previous"] >= 1).astype("int8")
    d["engagement_score"] = (
        d["previous_ge1"] + d["pdays_1_5"] + d["dur_high"] + (d["poutcome"] == 3).astype("int8")
    ).astype("int8")
    d["fatigue_score"] = (
        (d["campaign"] >= 5).astype("int8") + (d["campaign"] >= 8).astype("int8") +
        d["dur_low"] + d["pdays_never"]
    ).astype("int8")

    out = pd.DataFrame(index=d.index)
    out["dur_low"] = d["dur_low"]
    out["dur_mid"] = d["dur_mid"]
    out["dur_high"] = d["dur_high"]
    out["month_summer"] = d["month_summer"]
    out["month_fall"] = d["month_fall"]
    out["month_year_end"] = d["month_year_end"]
    out["month_early"] = d["month_early"]
    out["month_late_spring"] = d["month_late_spring"]
    out["engagement_low"] = (d["engagement_score"] <= 1).astype("int8")
    out["fatigue_low"] = (d["fatigue_score"] <= 1).astype("int8")
    out["housing"] = d["housing"].astype("int8")
    out["poutcome_success"] = (d["poutcome"] == 3).astype("int8")
    out["pdays_never"] = d["pdays_never"]
    return out

X_train_glass = engineer_features_bank(X_train)
X_test_glass = engineer_features_bank(X_test)

glass_X_train = glass_model.predict_proba(X_train_glass)[:, 1]
glass_X_test = glass_model.predict_proba(X_test_glass)[:, 1]

print(f"  Train: mean={glass_X_train.mean():.4f}, std={glass_X_train.std():.4f}")
print(f"  Test:  mean={glass_X_test.mean():.4f}, std={glass_X_test.std():.4f}")

# Stage 3: EBM
print("\n🔹 Stage 3: EBM")
ebm_X_train = ebm_model.predict_proba(X_train)[:, 1]
ebm_X_test = ebm_model.predict_proba(X_test)[:, 1]

print(f"  Train: mean={ebm_X_train.mean():.4f}, std={ebm_X_train.std():.4f}")
print(f"  Test:  mean={ebm_X_test.mean():.4f}, std={ebm_X_test.std():.4f}")

# Verify all predictions are on same samples
assert len(lr_X_train) == len(glass_X_train) == len(ebm_X_train)
assert len(lr_X_test) == len(glass_X_test) == len(ebm_X_test)
print(f"\n✅ All models have predictions on same {len(lr_X_train)} train / {len(lr_X_test)} test samples")

# --------------------------------------------------
# 3B. EVALUATE EACH STAGE MODEL
# --------------------------------------------------
print("\n" + "="*60)
print("📊 3B. STAGE-BY-STAGE PERFORMANCE METRICS")
print("="*60)

def evaluate_stage(y_true, y_pred_proba, stage_name, threshold=0.5):
    """Evaluate a stage model with comprehensive metrics"""
    y_pred_class = (y_pred_proba >= threshold).astype(int)

    acc = accuracy_score(y_true, y_pred_class)
    prec = precision_score(y_true, y_pred_class, zero_division=0)
    rec = recall_score(y_true, y_pred_class, zero_division=0)
    f1 = f1_score(y_true, y_pred_class, zero_division=0)
    roc = roc_auc_score(y_true, y_pred_proba)

    cm = confusion_matrix(y_true, y_pred_class)

    # Per-class metrics
    tn, fp, fn, tp = cm.ravel()

    # Class 0 (No Subscribe) metrics
    class0_precision = tn / (tn + fn) if (tn + fn) > 0 else 0
    class0_recall = tn / (tn + fp) if (tn + fp) > 0 else 0
    class0_f1 = 2 * (class0_precision * class0_recall) / (class0_precision + class0_recall) if (class0_precision + class0_recall) > 0 else 0

    # Class 1 (Subscribe) metrics
    class1_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    class1_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    class1_f1 = 2 * (class1_precision * class1_recall) / (class1_precision + class1_recall) if (class1_precision + class1_recall) > 0 else 0

    print(f"\n{'='*60}")
    print(f"📊 {stage_name}")
    print(f"{'='*60}")
    print(f"\n🎯 Overall Metrics:")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  ROC-AUC:   {roc:.4f}")

    print(f"\n📉 Class 0 (No Subscribe) - Support: {tn + fp}")
    print(f"  Precision: {class0_precision:.4f}  (TN={tn} / TN+FN={tn + fn})")
    print(f"  Recall:    {class0_recall:.4f}  (TN={tn} / TN+FP={tn + fp})")
    print(f"  F1-Score:  {class0_f1:.4f}")

    print(f"\n📈 Class 1 (Subscribe) - Support: {tp + fn}")
    print(f"  Precision: {class1_precision:.4f}  (TP={tp} / TP+FP={tp + fp})")
    print(f"  Recall:    {class1_recall:.4f}  (TP={tp} / TP+FN={tp + fn})")
    print(f"  F1-Score:  {class1_f1:.4f}")

    print(f"\n📊 Confusion Matrix:")
    print(f"                 Pred No    Pred Yes")
    print(f"  Actual No      {tn:6d}     {fp:6d}")
    print(f"  Actual Yes     {fn:6d}     {tp:6d}")

    return {
        'accuracy': acc,
        'roc_auc': roc,
        'class0_precision': class0_precision,
        'class0_recall': class0_recall,
        'class0_f1': class0_f1,
        'class1_precision': class1_precision,
        'class1_recall': class1_recall,
        'class1_f1': class1_f1,
        'confusion_matrix': cm
    }

# Evaluate each stage on TEST set
stage1_metrics = evaluate_stage(y_test, lr_X_test, "STAGE 1: LOGISTIC REGRESSION")
stage2_metrics = evaluate_stage(y_test, glass_X_test, "STAGE 2: GLASS-BRW (RF)")
stage3_metrics = evaluate_stage(y_test, ebm_X_test, "STAGE 3: EBM")

# Summary comparison table
print(f"\n" + "="*80)
print(f"📊 STAGE COMPARISON SUMMARY (TEST SET)")
print(f"="*80)
print(f"\n{'Metric':<25} {'LR':>12} {'GLASS-BRW':>12} {'EBM':>12}")
print(f"{'-'*65}")
print(f"{'Accuracy':<25} {stage1_metrics['accuracy']:>12.4f} {stage2_metrics['accuracy']:>12.4f} {stage3_metrics['accuracy']:>12.4f}")
print(f"{'ROC-AUC':<25} {stage1_metrics['roc_auc']:>12.4f} {stage2_metrics['roc_auc']:>12.4f} {stage3_metrics['roc_auc']:>12.4f}")
print(f"\n{'CLASS 0 (No Subscribe):':<25}")
print(f"{'  Precision':<25} {stage1_metrics['class0_precision']:>12.4f} {stage2_metrics['class0_precision']:>12.4f} {stage3_metrics['class0_precision']:>12.4f}")
print(f"{'  Recall':<25} {stage1_metrics['class0_recall']:>12.4f} {stage2_metrics['class0_recall']:>12.4f} {stage3_metrics['class0_recall']:>12.4f}")
print(f"{'  F1-Score':<25} {stage1_metrics['class0_f1']:>12.4f} {stage2_metrics['class0_f1']:>12.4f} {stage3_metrics['class0_f1']:>12.4f}")
print(f"\n{'CLASS 1 (Subscribe):':<25}")
print(f"{'  Precision':<25} {stage1_metrics['class1_precision']:>12.4f} {stage2_metrics['class1_precision']:>12.4f} {stage3_metrics['class1_precision']:>12.4f}")
print(f"{'  Recall':<25} {stage1_metrics['class1_recall']:>12.4f} {stage2_metrics['class1_recall']:>12.4f} {stage3_metrics['class1_recall']:>12.4f}")
print(f"{'  F1-Score':<25} {stage1_metrics['class1_f1']:>12.4f} {stage2_metrics['class1_f1']:>12.4f} {stage3_metrics['class1_f1']:>12.4f}")
print(f"="*80)

# --------------------------------------------------
# 4. CREATE META-FEATURES
# --------------------------------------------------
print("\n" + "="*60)
print("🔨 4. CREATING META-FEATURES")
print("="*60)

meta_train = pd.DataFrame({
    'lr_prob': lr_X_train,
    'rf_prob': glass_X_train,
    'ebm_prob': ebm_X_train
})

meta_test = pd.DataFrame({
    'lr_prob': lr_X_test,
    'rf_prob': glass_X_test,
    'ebm_prob': ebm_X_test
})

# Disagreement signals
meta_train['lr_rf_diff'] = np.abs(meta_train['lr_prob'] - meta_train['rf_prob'])
meta_train['lr_ebm_diff'] = np.abs(meta_train['lr_prob'] - meta_train['ebm_prob'])
meta_train['rf_ebm_diff'] = np.abs(meta_train['rf_prob'] - meta_train['ebm_prob'])

meta_test['lr_rf_diff'] = np.abs(meta_test['lr_prob'] - meta_test['rf_prob'])
meta_test['lr_ebm_diff'] = np.abs(meta_test['lr_prob'] - meta_test['ebm_prob'])
meta_test['rf_ebm_diff'] = np.abs(meta_test['rf_prob'] - meta_test['ebm_prob'])

# Confidence geometry
P_train = meta_train[['lr_prob', 'rf_prob', 'ebm_prob']].values
meta_train['max_confidence'] = P_train.max(axis=1)
meta_train['min_confidence'] = P_train.min(axis=1)
meta_train['std_confidence'] = P_train.std(axis=1)

P_test = meta_test[['lr_prob', 'rf_prob', 'ebm_prob']].values
meta_test['max_confidence'] = P_test.max(axis=1)
meta_test['min_confidence'] = P_test.min(axis=1)
meta_test['std_confidence'] = P_test.std(axis=1)

print(f"✅ Meta-features created: {meta_train.shape}")
print(f"   Features: {list(meta_train.columns)}")

# --------------------------------------------------
# 5. TRAIN META-EBM (10-FOLD CV)
# --------------------------------------------------
print("\n" + "="*60)
print("🎯 5. TRAINING META-EBM (10-FOLD CV)")
print("="*60)

meta_ebm = ExplainableBoostingClassifier(
    max_bins=16,
    max_interaction_bins=8,
    interactions=5,
    outer_bags=10,
    inner_bags=0,
    learning_rate=0.01,
    min_samples_leaf=5,
    max_leaves=3,
    random_state=42,
    n_jobs=-1
)

meta_ebm.fit(meta_train, y_train)
print("✅ Meta-EBM trained!")

# --------------------------------------------------
# 6. EVALUATE
# --------------------------------------------------
print("\n" + "="*60)
print("📊 6. META-EBM EVALUATION")
print("="*60)

meta_pred_test = meta_ebm.predict_proba(meta_test)[:, 1]
meta_pred_test_class = (meta_pred_test >= 0.5).astype(int)

accuracy = accuracy_score(y_test, meta_pred_test_class)
precision = precision_score(y_test, meta_pred_test_class)
recall = recall_score(y_test, meta_pred_test_class)
f1 = f1_score(y_test, meta_pred_test_class)
roc_auc = roc_auc_score(y_test, meta_pred_test)

print(f"\n🎯 META-EBM PERFORMANCE:")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")
print(f"  ROC-AUC:   {roc_auc:.4f}")

cm = confusion_matrix(y_test, meta_pred_test_class)
print(f"\nConfusion Matrix:\n{cm}")
print(f"\nClassification Report:")
print(classification_report(y_test, meta_pred_test_class,
                          target_names=['No Subscribe', 'Subscribe']))

# --------------------------------------------------
# 7. SAVE CHECKPOINT
# --------------------------------------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint = {
    'meta_ebm_model': meta_ebm,
    'stage_models': {
        'lr_model': lr_model,
        'glass_model': glass_model,
        'rf_model': rf_model,
        'ebm_model': ebm_model
    },
    'performance': {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc
    },
    'training_date': datetime.now().isoformat()
}

filename = f"/content/meta_ebm_cascade_checkpoint_{timestamp}.pkl"
with open(filename, 'wb') as f:
    pickle.dump(checkpoint, f)

print(f"\n✅ Saved: {filename}")
print("\n" + "="*80)
print("🎉 4-STAGE CASCADE COMPLETE!")
print("="*80)


🎯 META-EBM (STAGE 4) - FINAL CASCADE ASSEMBLY
✅ Class definitions loaded

📂 1. LOADING STAGE 1-3 MODELS
✅ LR model loaded
✅ GLASS-BRW model loaded (7 rules)
✅ EBM model loaded

🔧 2. CREATING SINGLE TRAIN/TEST SPLIT
✅ Single split created:
  Train: (36168, 16) samples
  Test:  (9043, 16) samples
  Train positives: 0.1170
  Test positives:  0.1170

🎲 3. GENERATING PREDICTIONS FROM ALL STAGES

🔹 Stage 1: Logistic Regression
  Train: mean=0.3306, std=0.2785
  Test:  mean=0.3316, std=0.2785

🔹 Stage 2: GLASS-BRW
  Train: mean=0.0993, std=0.1171
  Test:  mean=0.1002, std=0.1191

🔹 Stage 3: EBM
  Train: mean=0.1170, std=0.2092
  Test:  mean=0.1157, std=0.2064

✅ All models have predictions on same 36168 train / 9043 test samples

📊 3B. STAGE-BY-STAGE PERFORMANCE METRICS

📊 STAGE 1: LOGISTIC REGRESSION

🎯 Overall Metrics:
  Accuracy:  0.8236
  ROC-AUC:   0.8886

📉 Class 0 (No Subscribe) - Support: 7985
  Precision: 0.9685  (TN=6605 / TN+FN=6820)
  Recall:    0.8272  (TN=6605 / TN+FP=7985)
  F

In [ ]:
# ============================================================
# TODO: FINAL STAGE — GLASS-BOX ENSEMBLE (META LAYER)
# ============================================================
# NOTE: Upstream models (LR, GLASS-BRW, EBM) will be fine-tuned first.
#       This ensemble stage will be revisited and adjusted to reflect
#       updated constraints, thresholds, and feature semantics.

# TODO: Re-evaluate meta-model feature inputs to ensure alignment with
#       updated upstream outputs (probabilities, abstentions, confidence signals).
# TODO: Tune ensemble weighting and routing logic to optimize recall–precision
#       trade-offs under new upstream specifications.
# TODO: Introduce constraint-aware gating (e.g., minimum recall guards,
#       confidence thresholds, disagreement signals) to prevent overfitting
#       and preserve interpretability.
# TODO: Validate ensemble stability via cross-validation and stress tests
#       on edge cases where upstream models disagree or abstain.
# TODO: Ensure end-to-end glass-box consistency: every ensemble decision
#       must be decomposable into transparent upstream contributions.
# TODO: Verify that all model objects, classes, and methods are correctly
#       instantiated, serialized, and transferred across pipeline stages.
#       Current results suggest a potential alignment mismatch; this will
#       be diagnosed and resolved when returning to this stage.
